# QEI ASL CBF RATING TOOL

<!DOCTYPE html>
<html lang="en">

<body>
    <img src="https://pennbrain.upenn.edu/wp-content/uploads/2019/05/Upenn_Brain_Science_Center_Logo_400.png" alt="Description of the image" alt="Description of the first image" align="left" width="45%">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/92/UPenn_shield_with_banner.svg/1200px-UPenn_shield_with_banner.svg.png" alt="Description of the second image" align="left" width="20%">
</body>
</html>

This notebook is designed to facilitate the rating of a new ASL CBF dataset efficiently and interactively.

<div style="font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;">

<h2 style="color: #3465A4;">📋 USAGE INSTRUCTIONS</h2>

<p>To ensure optimal use of this tool, please adhere to the following guidelines:</p>

<ul>
<li><strong>Execution</strong>:  You can activate this tool either by running each code cell individually using the Play button on the left or by executing all cells automatically. We strongly recommend using the second option. For that, please follow the following steps:
<pre style="background-color: #EEE; padding: 8px;">
1. Navigate to the top menu of your Colab notebook.
2. Select "Runtime".
3. Choose "Run all" from the drop-down menu.
</pre>
We strongly recommend using the second option.
<p>After running all cells, a new window will be displayed. You must grant access to your Google Drive account. </p>

<p>Then, the needed directories will be created in case they do not exist. Moreover, the dataset will be downloaded from Dropbox and stored in your Google Drive.</p>

</li>
<li><strong>Interface and Rating Process</strong>:
<ol>
<li><strong>View Examination</strong>: Prior to rating, all image views must be examined. The rating buttons, found at the right of the figure, will be activated only after observing all view options. </li>
<li><strong>Saving Ratings</strong>: Click <em>Next Image</em> to save your rating in an Excel file. In order to do so, you must rate the image and select the most likely source of the artifact. After doing so, the "Next" button will be activated and then you will be able to save it. </li>
<li><strong>Previous Ratings</strong>: Click <em>Previous Image</em> to visualize or/and modify the previous image and its corresponding rating. </li>
<li><strong>Intensity scale</strong>: Click on the horizontal slide bar to change the min-max value of the image intensity scale. All the values above and below the min-max values, will be thresholded and changed to the corresponding value (min or max). The default min-max values are [-20,80].</li>
<li><strong>Changing the displayed slices</strong>: Click on the vertical slide bar to change the current slices that are displayed.</li>
<li><strong>Program Termination</strong>: You can terminate the rating process at any moment by pressing <em>Stop Program</em>. This action deactivates all buttons and concludes the session. </li>
<li><strong>Session Resumption</strong>: To continue rating additional images, simply re-execute all code cells following the instructions provided above. </li>
<li><strong>Downloading Ratings</strong>: Upon completing image ratings, a <em>Download Ratings</em> button will appear. Clicking this button allows you to download the dataset ratings to your computer. Alternatively, access the Excel file directly from your Google Drive at '<em>content/drive/My Drive/Rating_Results/Ratings.xlsx</em>'.</li>
</ol>
 <p> </p>
</li>
<li><strong>Ratings Criteria</strong>: To rate the images, the following criteria <strong>must be followed:</strong>
<ol>
<p><strong>-Excellent (rating 4): </strong> High quality CBF map without artifacts.</p>
<p><strong>-Average (rating 3): </strong> Acceptable quality CBF map with minor artifacts that do not significantly reduce information value.</p>
<p><strong>-Poor (rating 2): </strong> CBF map has one or more major artifacts, but can still potentially yield useful information.</p>
<p><strong>-Unacceptable (rating 1): </strong>CBF map is severely degraded by artifacts and is uninterpretable.</p>
</ol>
</li></li></ul>

<h2 style="color: #3465A4;">🔧 REQUIREMENTS</h2>

<p>This tool assumes you have a Google Drive account with adequate storage space for an Excel file containing your ratings and for the Dataset. Additionally, a strong internet connection is strictly needed. </p>

<h2 style="color: #3465A4;">ℹ️ MORE INFORMATION</h2>

<p>If you require any additional information or support, please do not hesitate to contact: <a href="mailto:xavier.urbano@pennmedicine.upenn.edu>">xavier.urbano@pennmedicine.upenn.edu</a>.</p>

> Colab created by [Xavier Beltran Urbano](https://github.com/xavibeltranurbano)

</div>


In [ ]:
# @title 1. Set up the directories and the dependencies
#@markdown Please execute this cell by pressing the _Play_ button
#@markdown on the left to install and import all the required libraries,
#@markdown and to create a folder in your google drive where the results (ratings)
#@markdown will be stored. The folder will be in the following directory:
#@markdown _'/content/drive/My Drive/Rating_Results'_


###### Set up the dependencies ######

!pip install pandas openpyxl xlsxwriter
import os
import io
import imageio.v2 as imageio
import cv2
import base64
import numpy as np
import nibabel as nib
import math
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import Javascript, display
from ipywidgets import Layout, Label, HBox, VBox, HTML,ButtonStyle
from google.colab import drive
from google.colab.patches import cv2_imshow
import requests
import zipfile
from io import BytesIO
import copy
import random

###### Create (in case is not created) a folder where to store the excel with the ratings ######

# This will prompt for authorization.
drive.mount('/content/drive')

# Path for the new folder
folder_path = '/content/drive/My Drive/Rating_Results'

# Create a folder if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print('Folder created:', folder_path)
else:
    print('Folder already exists:', folder_path)


###### Create (in case is not created) an excel file with the ratings ######

# Create a DataFrame
file_names=os.listdir("/content/drive/My Drive/Rating_Results")

if 'Ratings.xlsx' in file_names:
  print("File already exists.")
  # Start with the file after the last file rated in the excel
  df_read = pd.read_excel('/content/drive/My Drive/Rating_Results/Ratings.xlsx')
  currentFile=len(df_read)
else:
  df = pd.DataFrame(columns=['File Name', 'Rating', 'Artifacts'])
  folder_path = '/content/drive/My Drive/Rating_Results' # Replace with your folder name
  excel_file_name = 'Ratings.xlsx'
  excel_path = f'{folder_path}/{excel_file_name}'
  df.to_excel(excel_path, index=False)
  print(f'Excel file created at: {excel_path}')
  # Set current File as the first element
  currentFile=0

In [ ]:
# @title 2. Download Dataset
#@markdown Please execute this cell by pressing the _Play_ button
#@markdown on the left to download the dataset and store it in your Google Drive. The Dataset will be in the following directory:
#@markdown _'/content/drive/My Drive/Dataset_ASL'_


# Check if the Dataset directory exists, create if it does not
destination_dir='/content/drive/MyDrive/Dataset_ASL/'
direct_link="Put your dropbox link here"

# Ensure the destination directory exists
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
    print("Downloading file...")
    # Attempt to download the file
    response = requests.get(direct_link)
    if response.status_code == 200:
        zip_file = BytesIO(response.content)
        try:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                print("Unzipping file...")
                zip_ref.extractall(destination_dir)
            print("File downloaded and unzipped successfully!")
        except zipfile.BadZipFile:
            # Handle case where the file is not a valid zip file
            print("Downloaded file is not a valid zip file.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
   print("The Dataset is already stored in your Google Drive")



In [ ]:
# @title 3. Rating tool definition
#@markdown Please execute this cell by pressing the _Play_ button
#@markdown on the left to initialize the rating tool.

class RatingTool:
    """

     QEI ASL CBF Rating Tool.

    """
    def __init__(self,imagePath, excelPath,currentImagePosition):
        # Inject custom CSS for widget font sizes
        style = """
        <style>
            .jupyter-widgets { font-size: 20px !important; }
            .widget-label { font-size: 20px !important; }
            .widget-radio-box label {margin-bottom: 10px; font-size:20px;}
        </style>
        """
        random.seed(48)
        display(HTML(style))
        self.imagePath=imagePath
        listNames=sorted(os.listdir(imagePath))
        self.fileNames=[os.path.join(imagePath, imageName) for imageName in listNames if '.DS_Store' not in imageName]
        random.shuffle(self.fileNames)
        self.fileNames = [fileName for fileName in self.fileNames if not fileName.endswith('.xlsx')]
        if currentImagePosition>=len(self.fileNames):
          print("The dataset is already rated.")
        else:
          self.currentImage=currentImagePosition
          self.excelPath=excelPath
          self.excelResults=pd.read_excel(excelPath)
          self.seenSagitalView=False
          self.seenCoronalView=False
          self.ratingValue=None
          self.artifactValue=None
          self.changingImage=False
          self.figureOutput = widgets.Output()
          self.readImageInit()
          self.channel=2
          self.min, self.max=-20,80
          self.progressText = HTML()
          self.previousImage= widgets.Button(description="Previous Image")
          self.updateProgressText()
          self.actualRating = HTML()
          self.actualArtifact = HTML(value='',ayout=Layout(width='10px', overflow='hidden'))
          self.artifact11=widgets.Text(value=None, placeholder='Enter another artifact source.',description='Others:',disabled=False,layout=Layout(width='300px', height='50px', padding='10px'))
          self.NamesSlices=pd.read_excel(f"{imagePath}/image_slices.xlsx")
          self.rangeSlices=0
          self.middleSlices=True
          self.numberSlices_Axial, self.numberSlices_Coronal, self.numberSlices_Sagittal, self.numberSlices_All_Axial= self.NamesSlices.loc[self.NamesSlices['File_Name'] == self.fileNames[self.currentImage].split("/")[-1], ['nSlices_axial_No_Blank', 'nSlices_coronal', 'nSlices_sagittal','nSlices_axial_total']].iloc[0]
          self.display=True
          self.updateActualRating()
          self.execute()


    # Function to normalize image
    def scale_image(self,image,image_min,image_max):
        # Normalize the image following min-max strategy
        image_scaled=image.copy()
        image_scaled[image_scaled<image_min]=image_min
        image_scaled[image_scaled>image_max]=image_max
        return np.trunc(image_scaled)

    # Preprocess Image
    def preprocessImage(self,imagePath, min=-20, max=80):
        data= imageio.imread(imagePath)
        data_scaled = self.scale_image(data, min, max)
        return data_scaled

    # Read Initial Images
    def readImageInit(self):
        allImages=[]
        listIter=[]
        if self.currentImage>=1 and (self.currentImage+1)!=len(self.fileNames): listIter=[-1,0,+1]
        elif (self.currentImage+1)==len(self.fileNames): listIter=[-1,0,None]
        else: listIter=[None,0,+1]
        for iter in listIter:
            image=[]
            if iter==None: allImages.append(image)
            else:
                img_path = self.fileNames[self.currentImage+iter]
                imageName = img_path.split("/")[-1]
                for view in ['_sagittal.tiff','_coronal.tiff','_axial.tiff']:
                    img_path_new = os.path.join(img_path,"CBF_Map"+view )
                    image.append(self.preprocessImage(img_path_new))
                allImages.append(image)
        self.imageArrayFigures=allImages
        self.originalImage=copy.deepcopy(self.imageArrayFigures[1])

    # Read Next or Previous Image and modify the batch
    def readImageBatch(self, next):
        self.imageArrayFigures[1]=copy.deepcopy(self.originalImage)
        if next:
            self.imageArrayFigures.pop(0)
            if (self.currentImage+1)==len(self.fileNames): img_path = self.fileNames[self.currentImage]
            else: img_path = self.fileNames[self.currentImage+1]
        else:
            self.imageArrayFigures.pop(-1)
            img_path = self.fileNames[self.currentImage-1]
        image=[]
        imageName = img_path.split("/")[-1]
        for view in ['_sagittal.tiff','_coronal.tiff','_axial.tiff']:
            img_path_new = os.path.join(img_path,"CBF_Map"+view )
            image.append(self.preprocessImage(img_path_new))

        if next: self.imageArrayFigures.append(image)
        else: self.imageArrayFigures.insert(0,image)

    # Artifact Button Definitions
    def defineButtonsArtifacts(self):
        # Custom styles for different buttons with muted colors
        artifact_button_style = ButtonStyle(button_color='#ADADAD')  # Light Slate Gray
        custom_artifact_layout = Layout(width='300px', height='50px', padding='10px')  # Adjust sizes as needed
        self.artifact0=widgets.Button(description="Free of Artifact", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact1=widgets.Button(description="Motion Artifact", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact2=widgets.Button(description="Transit Artifact", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact3=widgets.Button(description="Improbable CBF values", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact4=widgets.Button(description="Low SNR", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact5=widgets.Button(description="Probable labeling asymmetry", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact6=widgets.Button(description="Fat shift artifact", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact7=widgets.Button(description="Slice timing correction error", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact8=widgets.Button(description="Image Distortion", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact9=widgets.Button(description="Poor Resolution", style=artifact_button_style, layout=custom_artifact_layout)
        self.artifact10=widgets.Button(description="Clipping artifact", style=artifact_button_style, layout=custom_artifact_layout)

    # Button Definitions
    def defineButtons(self):
        # Custom styles for different buttons with muted colors
        stop_program_style = ButtonStyle(button_color='#B45E5E')  # Dark red
        rating_button_style = ButtonStyle(button_color='#ADADAD')  # Light Slate Gray
        previous_image_style = ButtonStyle(button_color='#5EAAB4')  # Golden Rod
        view_button_style = ButtonStyle(button_color='#DAA520')  # Gray
        next_button_style = ButtonStyle(button_color='#55A255')  # Dark Green
        # Custom layout to increase button size, indirectly increasing perceived font size
        custom_layout = Layout(width='250px', height='50px', padding='10px')  # Adjust sizes as needed
        # Rating Buttons with custom style and layout
        self.rating1 = widgets.Button(description="1 (Unacceptable)", style=rating_button_style, layout=custom_layout)
        self.rating2 = widgets.Button(description="2 (Poor)", style=rating_button_style, layout=custom_layout)
        self.rating3 = widgets.Button(description="3 (Average)", style=rating_button_style, layout=custom_layout)
        self.rating4 = widgets.Button(description="4 (Excellent)", style=rating_button_style, layout=custom_layout)
        # View Buttons with custom style and layout
        self.axialView = widgets.Button(description="Axial View", style=view_button_style, layout=custom_layout)
        self.sagittalView = widgets.Button(description="Sagittal View", style=view_button_style, layout=custom_layout)
        self.coronalView = widgets.Button(description="Coronal View", style=view_button_style, layout=custom_layout)
        # Stop and Next Buttons with custom style and layout
        self.stopProgram = widgets.Button(description="Stop Program", style=stop_program_style, layout=custom_layout)
        self.nextImage = widgets.Button(description="Next Image", style=next_button_style, layout=custom_layout)
        self.previousImage = widgets.Button(description="Previous Image", style=previous_image_style, layout=custom_layout)
        # Artifact buttons definition
        self.defineButtonsArtifacts()
        # Normalization Scale
        self.normScaleWidgets=widgets.RadioButtons(options=['[-20,80]', '[0,80]', 'Another Scale'],disabled=False)
        self.normScale=widgets.FloatRangeSlider(value=[-20, 80],min=-100, max=200,step=5,disabled=True,
                                              continuous_update=False,orientation='horizontal',readout=True, readout_format='d')
        self.slicesScroll=widgets.IntSlider(value=math.ceil(self.numberSlices_Axial/3),min=3, max=math.ceil(self.numberSlices_Axial/3), step=1,orientation='vertical', continuous_update=False,readout=False, layout=widgets.Layout(height='700px'))
        self.normScale.disabled=True



    # Artifact Button Action Definitions
    def defineArtifactsAction(self):
        artifact_labels = ["Free of Artifacts", "Motion artifact", "Transit artifact", "Improbable CBF values", "Low SNR", "Probable labeling asymmetry",
            "Fat shift artifact", "Slice timing correction error", "Image distortion", "Poor resolution", "Clipping artifact"]
        # Define all the artifact actions
        for i, label in enumerate(artifact_labels):
            getattr(self, f'artifact{i}').on_click(lambda b, label=label: self.artifactButtonClicked(value=label))
        self.artifact11.observe(lambda change: self.artifactButtonClicked(value=change['new']), names='value')


    # Artifact Button Action
    def artifactButtonClicked(self, value):
        if any(char.isalpha() for char in value.strip()):
            self.artifactValue=value
            self.updateActualRating()
            if self.ratingValue not in [None, 0] and self.artifactValue not in [None, 0]:
                self.nextImage.disabled = False
        else: self.nextImage.disabled = True


    # Button Action Definitions
    def defineButtonsAction(self):
        # · Rating Buttons
        for i in range(1, 5):
            getattr(self, f'rating{i}').on_click(lambda b, i=i: self.ratingButtonClicked(value=i))
        # · View Buttons
        self.axialView.on_click(lambda b: self.viewButtonClicked(channel=2))
        self.sagittalView.on_click(lambda b: self.viewButtonClicked(channel=0))
        self.coronalView.on_click(lambda b: self.viewButtonClicked(channel=1))
        # · Stop, Next and Previous Buttons
        self.stopProgram.on_click(lambda b: self.stopProgramClicked())
        self.nextImage.on_click(lambda b: self.nextImageClicked())
        self.previousImage.on_click(lambda b: self.previousImageClicked())
        # · Artifacts Buttons
        self.defineArtifactsAction()
        # Slide bar Action
        self.normScaleWidgets.observe(self.radioButtonChanges, names='value')
        self.normScale.observe(self.slideBarChanges, names='value')
        self.slicesScroll.observe(self.slicesScrollChanges, names='value')


    # Radio Button Change Action
    def radioButtonChanges(self,change):
        if change['new']=='[0,80]':
            self.min,self.max=0,80
            self.updateImage()
            self.normScale.disabled=True
        elif change['new']=='[-20,80]':
            self.min,self.max=-20,80
            self.updateImage()
            self.normScale.disabled=True
        else:
            self.normScale.disabled=False


    # Update Image to tehe selected Scale
    def updateImage(self):
        originalCopy = [self.originalImage[0].copy(),self.originalImage[1].copy(),self.originalImage[2].copy()]
        for iter, image in enumerate(originalCopy):
            data_scaled= self.scale_image(image,image_min=self.min, image_max=self.max)
            self.imageArrayFigures[1][iter]=data_scaled
        if self.ratingValue!=0 or self.changingImage==False:
            self.displayFigure(iterImage=1)


    # Slide Bar Change Actions
    def slideBarChanges(self, change):
        self.min, self.max=change['new']
        self.updateImage()


    # Save the rating and the name in the excel
    def saveRating(self):
        if self.fileNames[self.currentImage].split("/")[-1] in self.excelResults['File Name'].values: # Check if the file already exists
            self.excelResults.loc[self.excelResults['File Name'] == self.fileNames[self.currentImage].split("/")[-1], 'Rating'] = int(self.ratingValue)
            self.excelResults.loc[self.excelResults['File Name'] == self.fileNames[self.currentImage].split("/")[-1], 'Artifacts'] = self.artifactValue
        else:
            new_data = pd.DataFrame({'File Name': [self.fileNames[self.currentImage].split("/")[-1]], 'Rating': [int(self.ratingValue)], 'Artifacts': [self.artifactValue]})
            self.excelResults = pd.concat([self.excelResults, new_data], ignore_index=True)
        self.excelResults.to_excel(self.excelPath, index=False)


    # Rating Buttons Action
    def ratingButtonClicked(self, value):
        self.ratingValue=value
        self.updateActualRating()
        if self.ratingValue not in [None, 0] and self.artifactValue not in [None, 0]:
            self.nextImage.disabled = False


    # Update displayed components
    def displayUIComponents(self,iterImage):
        rating_description = HTML('<b>Image Rating:</b>')
        artifacts_description = HTML('<b>Artifact Source:</b>')
        layout_center = Layout(display='flex', justify_content='center', align_items='center')
        scale_description=HBox([ HTML('<b> Scale Values [min - max]:</b>')], layout=Layout(padding='150px 0px 0px 0px'))
        actualImage=HBox([self.progressText], layout=Layout(padding='0px 0px 0px 340px'))
        views_with_progress = HBox([self.sagittalView, self.coronalView, self.axialView ,actualImage], layout=Layout(padding='0px 250px 0px 0px'))
        actualRating=HBox([self.actualRating], layout=Layout(padding='20px 0px 0px 0px'))
        actualArtifact=HBox([self.actualArtifact], layout=Layout(padding='20px 0px 0px 0px'))
        normScale=HBox([self.normScale], layout=Layout(padding='10px 0px 0px 0px'))
        normScaleWidgets=HBox([self.normScaleWidgets], layout=Layout(padding='10px 0px 0px 0px'))
        ratings=VBox([rating_description, self.rating1, self.rating2, self.rating3, self.rating4, actualRating,scale_description,normScaleWidgets,normScale],
                    layout=Layout(display='flex', justify_content='center', align_items='center',padding='0px 0px 0px 90px'))
        artifacts=VBox([artifacts_description,  self.artifact0,self.artifact1,self.artifact2,self.artifact3,self.artifact4,self.artifact5,self.artifact6,self.artifact7,
                         self.artifact8,self.artifact9,self.artifact10,self.artifact11, actualArtifact],layout=Layout(display='flex', justify_content='center', align_items='center',padding='0px 0px 0px 45px'))
        ratings_and_image = HBox([self.figureOutput, self.slicesScroll,ratings, artifacts], layout=layout_center)
        next = HBox([self.nextImage], layout=Layout(padding='0px 450px 0px 0px'))
        previous=HBox([self.previousImage], layout=Layout(padding='0px 402px 0px 0px'))
        others=HBox([previous, next], layout=Layout(padding='0px 266px 0px 0px'))
        stop=HBox([self.stopProgram], layout=Layout(padding='50px 720px 0px 0px'))
        ui_components = VBox([views_with_progress, ratings_and_image, others, stop], layout=layout_center)
        if self.ratingValue is None:
            display(ui_components)
            self.displayFigure(iterImage)


    # Status Previous Button
    def statusPreviousButton(self, status):
        self.previousImage.disabled = status


    # Update the counter of rated images
    def updateProgressText(self):
        self.progressText.value = f'<b>Image:</b> {self.currentImage + 1} / {len(self.fileNames)}'
        if (self.currentImage)>0: self.previousImage.disabled = False
        else: self.previousImage.disabled = True


    # Update actual rating
    def updateActualRating(self):
        listNames = self.excelResults['File Name'].tolist()
        if self.ratingValue not in [None,0] or self.artifactValue not in [None,0]:
            if self.ratingValue not in [None,0] and self.artifactValue not in [None,0]:
                self.actualRating.value = f'<b>Rating Value:</b> {self.ratingValue}'
                self.actualArtifact.value = f'''<div style="max-width: 300px; word-wrap: break-word; overflow-wrap: break-word;">
                <b>Artifact Source:</b> {self.artifactValue}</div>'''
            elif self.ratingValue not in [None,0] and self.artifactValue in [None,0]:
                self.actualRating.value = f'<b>Rating Value:</b> {self.ratingValue}'
                self.actualArtifact.value = '<b>Artifact Source:</b> Not selected'
            else:
                self.actualRating.value = '<b>Rating Value:</b> Not rated'
                self.actualArtifact.value = f'''<div style="max-width: 300px; word-wrap: break-word; overflow-wrap: break-word;">
                 <b>Artifact Source:</b> {self.artifactValue}</div>'''
        else:
            if self.fileNames[self.currentImage].split("/")[-1] in listNames:
                actualRatingValue = self.excelResults.loc[self.excelResults['File Name'] == self.fileNames[self.currentImage].split("/")[-1], 'Rating'].values[0]
                actualArtifactValue = self.excelResults.loc[self.excelResults['File Name'] == self.fileNames[self.currentImage].split("/")[-1], 'Artifacts'].values[0]
                self.actualRating.value = f'<b>Rating Value:</b> {actualRatingValue}'
                self.ratingValue = actualRatingValue
                self.actualArtifact.value = f'''<div style="max-width: 300px; word-wrap: break-word; overflow-wrap: break-word;">
                <b>Artifact Source:</b> {actualArtifactValue} </div>'''
                self.artifactValue = actualArtifactValue
            else:
                self.actualRating.value = '<b>Rating Value:</b> Not rated'
                self.actualArtifact.value = '<b>Artifact Source:</b> Not selected'


    # Download Button Actiona
    def downloadButtonClicked(self,b):
        output = io.BytesIO()
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            self.excelResults.to_excel(writer, sheet_name='Results')
        output.seek(0)
        encoded = base64.b64encode(output.read()).decode()
        # Trigger a download link using JavaScript
        js_download = f"""
        var link = document.createElement('a');
        link.href = 'data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{encoded}';
        link.download = 'Rating_Results.xlsx';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
        """
        display(Javascript(js_download))


    # Disable all artifact buttons
    def disableButtonsArtifact(self):
        for i in range(12):
            getattr(self, f'artifact{i}').disabled = True

    # Slices Scrolling Changes
    def slicesScrollChanges(self, change):
        if not self.middleSlices:
            if self.channel==2:
                self.rangeSlices=abs(change['new']-math.ceil(self.numberSlices_Axial/3))
            else:
                self.rangeSlices=abs(change['new']-self.nRows)
            if self.display: self.displayFigure(iterImage=1)

    # Update values of the scrolling bar
    def updateScrollBar(self,display=True):
        self.rangeSlices=0
        if self.channel==2:
            self.slicesScroll.max,self.slicesScroll.min,self.slicesScroll.value=math.ceil(self.numberSlices_Axial/3), 3,math.ceil(self.numberSlices_Axial/3)
        else:
            slices=self.numberSlices_Coronal
            if self.channel==0: slices=self.numberSlices_Sagittal
            self.nRows = self.slicesScroll.max = self.slicesScroll.value=(self.imageArrayFigures[1][self.channel].shape[0]-(math.ceil(slices/3)*10))//(math.ceil(self.numberSlices_All_Axial))
            self.slicesScroll.min=4

    # Previous Image Button Action
    def previousImageClicked(self):
        self.min, self.max=-20,80
        self.channel=2
        self.currentImage -= 1
        self.numberSlices_Axial, self.numberSlices_Coronal, self.numberSlices_Sagittal, self.numberSlices_All_Axial= self.NamesSlices.loc[self.NamesSlices['File_Name'] == self.fileNames[self.currentImage].split("/")[-1], ['nSlices_axial_No_Blank', 'nSlices_coronal', 'nSlices_sagittal','nSlices_axial_total']].iloc[0]
        self.display=False
        self.updateScrollBar()
        self.display=True
        self.middleSlices=True
        self.displayFigure(iterImage=0)
        self.middleSlices=False
        self.seenSagitalView = self.seenCoronalView = False
        self.ratingValue = self.artifactValue = 0
        self.changingImage=self.normScale.disabled=True
        self.normScale.value=(self.min,self.max)
        self.normScaleWidgets.value = '[-20,80]'
        self.changingImage=False
        self.artifact11.value = ''
        self.updateProgressText()
        self.updateActualRating()
        self.disableButtonsRating()
        self.disableButtonsArtifact()
        self.readImageBatch(next=False)
        self.originalImage=copy.deepcopy(self.imageArrayFigures[1])

    # Next Button Action
    def nextImageClicked(self):
        if self.ratingValue not in [None, 0]:
            self.saveRating()
            self.currentImage += 1
            self.channel=2
            if self.currentImage < len(self.fileNames):
                self.min, self.max=-20,80
                self.numberSlices_Axial, self.numberSlices_Coronal, self.numberSlices_Sagittal, self.numberSlices_All_Axial= self.NamesSlices.loc[self.NamesSlices['File_Name'] == self.fileNames[self.currentImage].split("/")[-1], ['nSlices_axial_No_Blank', 'nSlices_coronal', 'nSlices_sagittal','nSlices_axial_total']].iloc[0]
                self.display=False
                self.updateScrollBar()
                self.display=True
                self.middleSlices=True
                self.displayFigure(iterImage=2)
                self.middleSlices=False
                self.seenSagitalView = self.seenCoronalView = False
                self.ratingValue = self.artifactValue = 0
                self.disableButtonsRating()
                self.changingImage=self.normScale.disabled=True
                self.normScale.value=(self.min,self.max)
                self.normScaleWidgets.value = '[-20,80]'
                self.changingImage=False
                self.updateProgressText()
                self.updateActualRating()
                self.artifact11.value = ''
                self.disableButtonsRating()
                self.disableButtonsArtifact()
                self.readImageBatch(next=True)
                self.originalImage=copy.deepcopy(self.imageArrayFigures[1])
            else:
                self.disableAll()
                print(f"\n{'-'*400}")
                print("\nAll images have been rated! Thank you so much for your effort, we really appreciate it.")
                #  Display download button
                # custom_layout = Layout(width='250px', height='50px', padding='10px')
                # download_button = widgets.Button(description="Download Ratings",layout=custom_layout)
                # download_button.on_click(self.downloadButtonClicked)
                # display(download_button)

    # Enable all interactive buttons
    def enableButtonsRating(self):
        for i in range(1,5):
            getattr(self, f'rating{i}').disabled = False
        if self.currentImage<len(self.fileNames) and self.ratingValue not in [None, 0]:
            self.nextImage.disabled = False


    # Enable all artifact buttons
    def enableButtonsArtifact(self):
        for i in range(12):
            getattr(self, f'artifact{i}').disabled = False


    # Disable all buttons
    def disableAll(self):
        for i in range(1,5):
            getattr(self, f'rating{i}').disabled = True

        self.normScaleWidgets.disabled = True
        self.normScale.disabled = True
        self.nextImage.disabled=True
        self.previousImage.disabled = True
        self.axialView.disabled = True
        self.sagittalView.disabled = True
        self.coronalView.disabled = True
        self.stopProgram.disabled=True


    # Disable all rating buttons
    def disableButtonsRating(self):
        for i in range(1,5):
            getattr(self, f'rating{i}').disabled = True
        # Modify Next Button status
        if self.ratingValue in [None, 0]: self.nextImage.disabled = True
        else: self.nextImage.disabled = False
        # Modify Previous Button status
        if (self.currentImage)>0: self.previousImage.disabled = False
        else: self.previousImage.disabled = True


    # Stop program
    def stopProgramClicked(self):
        self.disableAll()
        print(f"\n{'-'*400}")
        print("\nProgram stopped. No further actions can be performed. See you soon!")


    # View Buttons Action
    def viewButtonClicked(self,channel):
        self.channel=channel
        if channel==0: self.seenSagitalView=True
        else: self.seenCoronalView=True
        if self.seenSagitalView==True and self.seenCoronalView==True:
            self.enableButtonsRating()
            self.enableButtonsArtifact()
        self.display=False
        self.updateScrollBar()
        self.display=True
        self.middleSlices=True
        self.displayFigure(iterImage=1)
        self.middleSlices=False

    # Crop Image with the corresponding row
    def cropImage(self,iterImage):
        if self.channel==2:
            return self.imageArrayFigures[iterImage][self.channel][int(96*self.rangeSlices):int(96*self.rangeSlices+ 96*3),:]
        elif self.channel==1:
            return self.imageArrayFigures[iterImage][self.channel][int((self.numberSlices_All_Axial+10)*self.rangeSlices):int((self.numberSlices_All_Axial+10)*self.rangeSlices+ (self.numberSlices_All_Axial+10)*4),:]
        elif self.channel==0:
            return self.imageArrayFigures[iterImage][self.channel][int((self.numberSlices_All_Axial+10)*self.rangeSlices):int((self.numberSlices_All_Axial+10)*self.rangeSlices+ (self.numberSlices_All_Axial+10)*4),:]


    # Display the Figure
    def displayFigure(self, iterImage):
        with self.figureOutput:
            self.figureOutput.clear_output(wait=True)
            if self.middleSlices: self.rangeSlices, self.slicesScroll.value=math.ceil((self.slicesScroll.max-3)/2), (self.slicesScroll.max - math.ceil((self.slicesScroll.max-3)/2))
            img=self.cropImage(iterImage)
            self.middleSlices=False
            img = np.pad(img, (5, 5), 'constant')
            resized_image = cv2.resize(img, (1250, 1150), interpolation=cv2.INTER_LINEAR)
            dpi = 100
            fig_width, fig_height = 1250 / dpi, 1150 / dpi  # Convert pixels to inches
            fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=dpi)
            if self.max>0 and self.min<0:
                cax = ax.imshow(resized_image, cmap='gray', vmin=self.min, vmax=self.max)
            elif self.max<0 and self.min<0:
                cax = ax.imshow(resized_image, cmap='gray', vmin=self.min, vmax=0)
            elif self.max>0 and self.min>=0:
                cax = ax.imshow(resized_image, cmap='gray', vmin=0, vmax=self.max)
            cb = fig.colorbar(cax, ax=ax, fraction=0.0432, pad=0.02, location='left')
            tick_values = [self.min, self.max]
            tick_values.append(0)
            intermediate_ticks = np.linspace(np.min(tick_values), np.max(tick_values), num=10)
            tick_values = np.unique(np.concatenate((tick_values, intermediate_ticks)))
            cb.set_ticks(tick_values)
            cb.set_ticklabels([f'{int(np.trunc(v))}' for v in tick_values])
            ax.set_xticks([])
            ax.set_yticks([])
            plt.show()

    # Main code
    def execute(self):
        self.defineButtons()
        self.disableButtonsRating()
        self.disableButtonsArtifact()
        self.defineButtonsAction()
        self.displayUIComponents(iterImage=1)


In [ ]:
# @title 4. Execute the tool
#@markdown Please execute this cell by pressing the _Play_ button
#@markdown on the left to use the rating tool.

pathDataset="/content/drive/MyDrive/Dataset_ASL/QEI-Dataset_tiff"
pathExcel='/content/drive/MyDrive/Rating_Results/Ratings.xlsx'
ratingTool= RatingTool(imagePath=pathDataset,excelPath=pathExcel, currentImagePosition=currentFile)